In [4]:
from dotenv import load_dotenv
load_dotenv("/mnt/shared_disk/.env")

True

In [5]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["/home/bws/xm2025/git2025/langchian_test/examples/01_helloworld/server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [6]:
agent_response

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='da7c7bc1-4d63-437a-a109-e31253946835'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_F8LWzC0Unw6VMIZVZ6uV1OkP', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 77, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-4o', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a81bada0-8f28-4794-9a46-600d5a8842c6-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_F8LWzC0Unw6VMIZVZ6uV1OkP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 17, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='8', name='add', id='d2a38363-b3bf-4711-8318-6

In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

async with MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/home/bws/xm2025/git2025/langchian_test/examples/01_helloworld/server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://172.28.140.214:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    # math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
    weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

Traceback (most recent call last):
  File "/home/bws/miniconda3/envs/langgraph/lib/python3.12/site-packages/mcp/client/sse.py", line 53, in sse_client
    event_source.response.raise_for_status()
  File "/home/bws/miniconda3/envs/langgraph/lib/python3.12/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '502 Bad Gateway' for url 'http://localhost:8000/sse'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502

During handling of the above exception, another exception occurred:

  + Exception Group Traceback (most recent call last):
  |   File "/home/bws/miniconda3/envs/langgraph/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_1068240/2394221810.py", line 7, in <module>
  |     async with MultiServerMCPClient(


In [ ]:
# math_response

In [ ]:
weather_response